In [ ]:
import SDDP, JuMP, ASDDiP, PyPlot

In [ ]:
function Q(m,t,ms, x)
    sp = SDDP.getsubproblem(m,t,ms)
    vf = SDDP.valueoracle(sp)
    _, y_cont = SDDP.processvaluefunctiondata(vf, true, x)
    y_ald = ASDDiP.estimatevf(sp, x)
    maximum(hcat(y_cont, y_ald), 2)[:,1]
end

# A toy model for ALD-SDDiP

## Description

$$\begin{array}{rl}
      \min  & \mathbb{E}\left[\sum\limits_t |x_t|\right] \\
\text{s.t.} & \quad x_t = x_{t-1} + c_t + \xi_t \\
            & \quad c_t \in {\pm 1}
\end{array} $$

# Using only Strenghtened benders cuts:

This corresponds to $\rho = 0$, for all stages and iterations.

In [ ]:
ramp_mode = :None

In [ ]:
include("control.jl")

In [ ]:
ts = -3:0.02:3
for t = 1:7
    PyPlot.plot(ts, Q(m,t,1,ts), label="$t")
end
PyPlot.legend(title="Stage")
PyPlot.title("Future cost functions")
PyPlot.grid();

# Using ALD

We estimate $\displaystyle Lip_t = \beta^{t-1} + \beta^t + \ldots + \beta^{8-1} = \beta^{t-1}(1 + \ldots + \beta^{8-t}) = \frac{1 - \beta^{8+1-t}}{1 - \beta}\beta^{t-1}$.

# First strategy: homothetic

At iteration $n$ and stage $t$, we set
$$\rho_t = \min\left(1, \max\left(0, \frac{n-15}{15}\right)\right) \cdot Lip_t. $$

That is:

- in the first 15 stages, $\rho_t = 0$;
- then increase $\rho_t$ until it reaches $Lip_t$ in 15 stages;
- and keep at $Lip_t$ until the end.

In [ ]:
ramp_mode = :simple

In [ ]:
include("control.jl")

### Graph of FCF

In [ ]:
ts = -3:0.02:3
for t = 1:7
    PyPlot.plot(ts, Q(m,t,1,ts), label="$t")
end
PyPlot.legend(title="Stage")
PyPlot.title("Future cost functions")
PyPlot.grid();
#PyPlot.show()

# Second strategy: parallel

At iteration $n$ and stage $t$, we set
$$\rho_t = \min\left(Lip_t, \max\left(0, \frac{n-15}{15}\right)\right). $$

That is:

- in the first 15 stages, $\rho_t = 0$;
- then increase with equal steps at all stages, stopping at $Lip_t$; (so that different stages "saturate" at different times)
- and keep at $Lip_t$ until the end.

In [ ]:
ramp_mode = :parallel

In [ ]:
include("control.jl")

### Graph of FCF

In [ ]:
ts = -3:0.02:3
for t = 1:7
    PyPlot.plot(ts, Q(m,t,1,ts), label="$t")
end
PyPlot.legend(title="Stage")
PyPlot.title("Future cost functions")
PyPlot.grid();
#PyPlot.show()

# Third strategy: double of parallel

At iteration $n$ and stage $t$, we set
$$\rho_t = \min\left(2 Lip_t, \max\left(0, \frac{n-15}{15}\right)\right). $$

That is:

- in the first 15 stages, $\rho_t = 0$;
- then increase with equal steps at all stages, stopping at $2 Lip_t$; (so that different stages "saturate" at different times)
- and keep at $2 Lip_t$ until the end.

In [ ]:
ramp_mode = :parallel2

In [ ]:
include("control.jl")

### Graph of FCF

In [ ]:
ts = -3:0.02:3
for t = 1:7
    PyPlot.plot(ts, Q(m,t,1,ts), label="$t")
end
PyPlot.legend(title="Stage")
PyPlot.title("Future cost functions")
PyPlot.grid();
#PyPlot.show()